In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install tensorflow keras numpy matplotlib opencv-python

In [ ]:
import os
import cv2
import numpy as np
from tensorflow.keras.models import load_model

# Paths to your datasets
train_image_dir = '/content/drive/MyDrive/yolov11/dataset/images/train'
val_image_dir = '/content/drive/MyDrive/yolov11/dataset/images/val'
test_image_dir = '/content/drive/MyDrive/yolov11/dataset/images/test'  # Assuming a test folder exists

# Path to save predictions
output_dir = '/content/drive/MyDrive/yolov11/predictions/'

# Create output directories for predictions
os.makedirs(output_dir, exist_ok=True)
os.makedirs(os.path.join(output_dir, 'train'), exist_ok=True)
os.makedirs(os.path.join(output_dir, 'val'), exist_ok=True)
os.makedirs(os.path.join(output_dir, 'test'), exist_ok=True)

# Function to load image paths
def load_image_paths(image_dir):
    return [os.path.join(image_dir, filename) for filename in os.listdir(image_dir)]

# Function to process images in batches
def predict_and_save_in_batches(model, image_paths, output_folder, batch_size=16, target_size=(320, 320)):
    num_images = len(image_paths)
    for i in range(0, num_images, batch_size):
        batch_paths = image_paths[i:i+batch_size]
        batch_images = []
        batch_filenames = []

        for img_path in batch_paths:
            filename = os.path.basename(img_path)
            image = cv2.imread(img_path)
            image = cv2.resize(image, target_size) / 255.0  # Normalize
            batch_images.append(image)
            batch_filenames.append(filename)

        batch_images = np.array(batch_images)

        # Predict on batch
        predictions = model.predict(batch_images)

        for j, pred in enumerate(predictions):
            # Scale prediction to [0, 255] for saving
            pred_resized = cv2.resize(pred.squeeze(), target_size)
            pred_image = (pred_resized * 255).astype(np.uint8)

            # Save prediction
            output_path = os.path.join(output_folder, batch_filenames[j])
            cv2.imwrite(output_path, pred_image)

    print(f"Predictions saved to {output_folder}")

# Load the trained model
model = load_model('/content/drive/MyDrive/unet_fish_segmentation.h5')

# Generate and save predictions for train, val, and test datasets
for split, image_dir in zip(['train', 'val', 'test'],
                            [train_image_dir, val_image_dir, test_image_dir]):
    output_folder = os.path.join(output_dir, split)
    image_paths = load_image_paths(image_dir)
    predict_and_save_in_batches(model, image_paths, output_folder)

print("Predictions generated and saved for all datasets.")


1/1 ━━━━━━━━━━━━━━━━━━━━ 18s 18s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━

In [ ]:
import os
from PIL import Image

# Define paths for input and output datasets
input_root = "/content/drive/MyDrive/yolov11/dataset/unet_dataset/images"  # Update this to your U-Net dataset root directory
output_root = "/content/drive/MyDrive/yolov11/dataset/unet_dataset_640/images"  # Update this to your output directory

# Ensure the output directory exists
os.makedirs(output_root, exist_ok=True)

# Subfolders to process
subfolders = ["train", "val", "test"]

# Resize dimensions
resize_dim = (640, 640)

# Function to resize images
def resize_images(input_dir, output_dir):
    os.makedirs(output_dir, exist_ok=True)
    for filename in os.listdir(input_dir):
        if filename.endswith(".png") or filename.endswith(".jpg"):  # Adjust extensions if necessary
            input_path = os.path.join(input_dir, filename)
            output_path = os.path.join(output_dir, filename)

            # Open and resize the image
            img = Image.open(input_path)
            img_resized = img.resize(resize_dim)
            img_resized.save(output_path)

# Process each subfolder (train, val, test)
for subfolder in subfolders:
    # Paths for images
    input_images_path = os.path.join(input_root, subfolder)
    output_images_path = os.path.join(output_root, subfolder)

    # Resize images
    print(f"Resizing images in {input_images_path}...")
    resize_images(input_images_path, output_images_path)

print("Resizing completed!")


Resizing images in /content/drive/MyDrive/yolov11/dataset/unet_dataset/images/train...
Resizing images in /content/drive/MyDrive/yolov11/dataset/unet_dataset/images/val...
Resizing images in /content/drive/MyDrive/yolov11/dataset/unet_dataset/images/test...
Resizing completed!
